In [1]:
#Importing libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import time
import json
import requests
import pandas as pd
import numpy as np

In [2]:
# Set options and initiate webdriver
url = 'https://www.fifa.com/fifa-world-ranking/ranking-table/men/'

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('ignore-certificate-errors')

browser = webdriver.Chrome(options=options, executable_path='venv/chromedriver')
browser.get(url)

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_9104/266471312.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='venv/chromedriver')


In [3]:
#get dates using CSS or JSON in <script> tag
#dates = browser.find_elements(By.CSS_SELECTOR,'button.ff-dropdown_dropupContentButton__WC4zi')

data = json.loads(browser.find_element(By.ID,'__NEXT_DATA__').get_attribute('innerHTML'))
dates = data['props']['pageProps']['pageData']['ranking']['dates']
list_date = []

for date in dates:
    dict_date = {}
    ID = date['id']
    Date = date['text']
    URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId='+date['id']
    dict_date.update({'id':ID, 'date':Date, 'url':URL})
    list_date.append(dict_date)

df_date = pd.DataFrame(list_date)
df_date.head()

,id,date,url
0,id13869,22 Dec 2022,https://www.fifa.com/fifa-world-ranking/men?da...
1,id13792,6 Oct 2022,https://www.fifa.com/fifa-world-ranking/men?da...
2,id13750,25 Aug 2022,https://www.fifa.com/fifa-world-ranking/men?da...
3,id13687,23 Jun 2022,https://www.fifa.com/fifa-world-ranking/men?da...
4,id13603,31 Mar 2022,https://www.fifa.com/fifa-world-ranking/men?da...


In [133]:
#number of paginated pages - just for testing
pages = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[3]/main/section[2]/div/div/div[2]/div/div/div/div/div[2]')
pages[0].text[-1]


'5'

In [5]:
df = pd.DataFrame(columns=['Date','Rank','Image','Team','Points','Prev Points','Incr/Decr'])

for index, row in df_date.iterrows():
    
    list_team = []
    next_page = 1
   
    print('Date:', row['date'])
     
    date_combobox = browser.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/main/section[1]/div/div/div[1]/div[2]/div/button/div/div[2]')
    browser.execute_script('arguments[0].click();',date_combobox)
   
    current_page = browser.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/main/section[1]/div/div/div[1]/div[2]/div/div/button['+str(index+1)+']')
    browser.execute_script('arguments[0].click();',current_page) 
    
    time.sleep(2)
  
    while True:
        #get all teams within 1 page 
        teams = browser.find_elements(By.CSS_SELECTOR,'tr.row_rankingTableFullRow__Y_A4i ')
        
        for j in range(0, len(teams)):
            team_facts = teams[j].find_elements(By.TAG_NAME,'td')
            dict_team ={'Date':        row['date'],
                        'Rank':        team_facts[0].text,
                        'Image':       team_facts[1].find_element(By.TAG_NAME,'img').get_attribute('src'),
                        'Team':        team_facts[2].text,
                        'Points':      team_facts[3].text,
                        'Prev Points': team_facts[4].text,
                        'Incr/Decr':   team_facts[5].text
                       }
            #print(dict_team['Rank'], dict_team['Team'], dict_team['Points'])
            
            list_team.append(dict_team)
        
        next_page = next_page + 1 
        
        try:
            next_paginated_page = browser.find_element(By.XPATH,
                                                      '/html/body/div[1]/div/div[3]/main/section[2]/div/div/div[2]/div/div/div/div/div[2]/div['+str(next_page)+']')
            browser.execute_script("arguments[0].click();" ,next_paginated_page)
       
        except NoSuchElementException: 
            
            break
        
    df = df.append(pd.DataFrame(list_team), ignore_index = True)

    
browser.close()
browser.quit()    


Date: 22 Dec 2022
Date: 6 Oct 2022
Date: 25 Aug 2022


KeyboardInterrupt: 

In [157]:
#correct the date format of Sept
df['Date'] = df['Date'].replace('Sept','Sep',regex=True)

#standardize the country name
df['Team'] = df['Team'].replace("Côte d'Ivoire","Ivory Coast", regex=True)
df['Team'] = df['Team'].replace('Türkiye','Turkey', regex=True)
df['Team'] = df['Team'].replace('Curaçao','Curacao', regex=True)
df['Team'] = df['Team'].replace('São Tomé e Príncipe','Sao Tome e Principe', regex=True)
df['Team'] = df['Team'].replace('St. Vincent and the Grenadines','St. Vincent/Grenadines', regex=True)
df['Team'] = df['Team'].replace('St. Vincent / Grenadines','St. Vincent/Grenadines', regex=True)
df['Team'] = df['Team'].replace('St Vincent and the Grenadines','St. Vincent/Grenadines', regex=True)

df.to_excel ('fifa.xlsx', index = None, header=True)   